# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff05e732df0>
├── 'a' --> tensor([[-0.0665, -0.9558,  1.4624],
│                   [ 0.8509,  0.3691, -0.1015]])
└── 'x' --> <FastTreeValue 0x7ff05e732d30>
    └── 'c' --> tensor([[ 0.0334,  0.5059,  0.4702,  0.5720],
                        [ 0.8247, -0.5728,  0.4577, -0.0853],
                        [ 2.5292, -0.7270,  1.0855, -0.7834]])

In [4]:
t.a

tensor([[-0.0665, -0.9558,  1.4624],
        [ 0.8509,  0.3691, -0.1015]])

In [5]:
%timeit t.a

60.9 ns ± 0.243 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff05e732df0>
├── 'a' --> tensor([[ 0.2941, -0.7874, -0.2402],
│                   [-0.9668, -0.4935, -0.8439]])
└── 'x' --> <FastTreeValue 0x7ff05e732d30>
    └── 'c' --> tensor([[ 0.0334,  0.5059,  0.4702,  0.5720],
                        [ 0.8247, -0.5728,  0.4577, -0.0853],
                        [ 2.5292, -0.7270,  1.0855, -0.7834]])

In [7]:
%timeit t.a = new_value

60.4 ns ± 0.0351 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0665, -0.9558,  1.4624],
               [ 0.8509,  0.3691, -0.1015]]),
    x: Batch(
           c: tensor([[ 0.0334,  0.5059,  0.4702,  0.5720],
                      [ 0.8247, -0.5728,  0.4577, -0.0853],
                      [ 2.5292, -0.7270,  1.0855, -0.7834]]),
       ),
)

In [10]:
b.a

tensor([[-0.0665, -0.9558,  1.4624],
        [ 0.8509,  0.3691, -0.1015]])

In [11]:
%timeit b.a

54.8 ns ± 0.0413 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8532,  0.1308, -0.1174],
               [-0.9405,  0.1659, -0.9350]]),
    x: Batch(
           c: tensor([[ 0.0334,  0.5059,  0.4702,  0.5720],
                      [ 0.8247, -0.5728,  0.4577, -0.0853],
                      [ 2.5292, -0.7270,  1.0855, -0.7834]]),
       ),
)

In [13]:
%timeit b.a = new_value

481 ns ± 0.186 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

829 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 22.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 767 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 297 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff030b17eb0>
├── 'a' --> tensor([[[-0.0665, -0.9558,  1.4624],
│                    [ 0.8509,  0.3691, -0.1015]],
│           
│                   [[-0.0665, -0.9558,  1.4624],
│                    [ 0.8509,  0.3691, -0.1015]],
│           
│                   [[-0.0665, -0.9558,  1.4624],
│                    [ 0.8509,  0.3691, -0.1015]],
│           
│                   [[-0.0665, -0.9558,  1.4624],
│                    [ 0.8509,  0.3691, -0.1015]],
│           
│                   [[-0.0665, -0.9558,  1.4624],
│                    [ 0.8509,  0.3691, -0.1015]],
│           
│                   [[-0.0665, -0.9558,  1.4624],
│                    [ 0.8509,  0.3691, -0.1015]],
│           
│                   [[-0.0665, -0.9558,  1.4624],
│                    [ 0.8509,  0.3691, -0.1015]],
│           
│                   [[-0.0665, -0.9558,  1.4624],
│                    [ 0.8509,  0.3691, -0.1015]]])
└── 'x' --> <FastTreeValue 0x7ff030b17730>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 48.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff05e732fd0>
├── 'a' --> tensor([[-0.0665, -0.9558,  1.4624],
│                   [ 0.8509,  0.3691, -0.1015],
│                   [-0.0665, -0.9558,  1.4624],
│                   [ 0.8509,  0.3691, -0.1015],
│                   [-0.0665, -0.9558,  1.4624],
│                   [ 0.8509,  0.3691, -0.1015],
│                   [-0.0665, -0.9558,  1.4624],
│                   [ 0.8509,  0.3691, -0.1015],
│                   [-0.0665, -0.9558,  1.4624],
│                   [ 0.8509,  0.3691, -0.1015],
│                   [-0.0665, -0.9558,  1.4624],
│                   [ 0.8509,  0.3691, -0.1015],
│                   [-0.0665, -0.9558,  1.4624],
│                   [ 0.8509,  0.3691, -0.1015],
│                   [-0.0665, -0.9558,  1.4624],
│                   [ 0.8509,  0.3691, -0.1015]])
└── 'x' --> <FastTreeValue 0x7ff05e7327f0>
    └── 'c' --> tensor([[ 0.0334,  0.5059,  0.4702,  0.5720],
                        [ 0.8247, -0.5728,  0.4577, -0.0853],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 67.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0334,  0.5059,  0.4702,  0.5720],
                       [ 0.8247, -0.5728,  0.4577, -0.0853],
                       [ 2.5292, -0.7270,  1.0855, -0.7834]],
              
                      [[ 0.0334,  0.5059,  0.4702,  0.5720],
                       [ 0.8247, -0.5728,  0.4577, -0.0853],
                       [ 2.5292, -0.7270,  1.0855, -0.7834]],
              
                      [[ 0.0334,  0.5059,  0.4702,  0.5720],
                       [ 0.8247, -0.5728,  0.4577, -0.0853],
                       [ 2.5292, -0.7270,  1.0855, -0.7834]],
              
                      [[ 0.0334,  0.5059,  0.4702,  0.5720],
                       [ 0.8247, -0.5728,  0.4577, -0.0853],
                       [ 2.5292, -0.7270,  1.0855, -0.7834]],
              
                      [[ 0.0334,  0.5059,  0.4702,  0.5720],
                       [ 0.8247, -0.5728,  0.4577, -0.0853],
                       [ 2.5292, -0.7270,  1.0855, -0.7834]],

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0334,  0.5059,  0.4702,  0.5720],
                      [ 0.8247, -0.5728,  0.4577, -0.0853],
                      [ 2.5292, -0.7270,  1.0855, -0.7834],
                      [ 0.0334,  0.5059,  0.4702,  0.5720],
                      [ 0.8247, -0.5728,  0.4577, -0.0853],
                      [ 2.5292, -0.7270,  1.0855, -0.7834],
                      [ 0.0334,  0.5059,  0.4702,  0.5720],
                      [ 0.8247, -0.5728,  0.4577, -0.0853],
                      [ 2.5292, -0.7270,  1.0855, -0.7834],
                      [ 0.0334,  0.5059,  0.4702,  0.5720],
                      [ 0.8247, -0.5728,  0.4577, -0.0853],
                      [ 2.5292, -0.7270,  1.0855, -0.7834],
                      [ 0.0334,  0.5059,  0.4702,  0.5720],
                      [ 0.8247, -0.5728,  0.4577, -0.0853],
                      [ 2.5292, -0.7270,  1.0855, -0.7834],
                      [ 0.0334,  0.5059,  0.4702,  0.5720],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 229 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
